In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl");

### Generate Data

In [2]:
function generate_Data(iterations, N, N_u, K, offdiag_sign)
    Input_Data = Dict()
    for iter in 1:iterations
        Input_Data["iter=$(iter)_Obs_Feat"] = rand(N_u);
        Input_Data["iter=$(iter)_A_true"] = rand(N,N_u);
        Input_Data["iter=$(iter)_B_true"] = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign);
        Input_Data["iter=$(iter)_P_dag"] = round.(rand(N, K),digits=2);
    end
    return Input_Data
end

generate_Data (generic function with 1 method)

### ETO 

In [3]:
function Run_ETO(iterations, N, N_u, K, Input_Data)
    RST_ETO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)
        RST_ETO["iter=$(iter)_Rev"] = rev_ETO
        RST_ETO["iter=$(iter)_Price"] = price_ETO
    end
    return RST_ETO
end

Run_ETO (generic function with 1 method)

### RO

In [4]:
function search_opt_price(N,p_lb,p_ub,b_n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, price[1:N])                      # y_{nk}
    @constraint(model, price .>= p_lb)
    @constraint(model, price .<= p_ub)
    @objective(model, Max,b_n' * price )
    optimize!(model)
    obj_val = objective_value(model)
    return obj_val
end


search_opt_price (generic function with 1 method)

In [ ]:
function Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef)
    RST_RO = Dict(); RST_RO_Two_Side = Dict()
    # psi_coef = 1
    # bd_coef = 0.000001
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        A_lb = A_true .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_true .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_true .- bd_coef .* abs.(ones(N,N));
        B_ub = B_true .+ bd_coef .* abs.(ones(N,N));

        
        p_ub = vec(maximum(P_dag,dims=2))
        p_lb = vec(minimum(P_dag,dims=2))
        p_max = maximum(p_ub)
        p_min = minimum(p_lb)
        Obs_Feat_Trun = [max(-Obs_Feat[ind],0) for ind in 1:N_u]
        psi_lb = zeros(N)
        for n in 1:N
            b_n = B_lb[n,:]
            obj_n = search_opt_price(N,p_lb,p_ub,b_n)
            psi_lb[n] = -exp(-Obs_Feat_Trun' * (A_ub[n,:] - A_lb[n,:]) + Obs_Feat' * A_lb[n,:] + obj_n)*(p_max-p_min)
        end
        
        # psi_lb = 0.0 - psi_coef * 10
        psi_ub = 0.0

        obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
        rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
        println("psi_lb = ",round.(psi_lb,digits=2),",rev_RO = ",round(rev_RO,digits=6),",price_RO = ",price_RO)
        RST_RO["iter=$(iter)_Rev"] = rev_RO
        RST_RO["iter=$(iter)_Price"] = price_RO
    end
    return RST_RO
end

In [ ]:
function Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef)
    RST_RO_Two_Side = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        A_lb = A_true .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_true .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_true .- bd_coef .* abs.(ones(N,N));
        B_ub = B_true .+ bd_coef .* abs.(ones(N,N));

        psi_lb = 0.0 - psi_coef * 10
        psi_ub = 0.0
        phi_lb = 0.0 - phi_coef * 10
        phi_ub = 0.0

        # Two-sided RO
        obj_RO_2,X_RO_2,time_RO_2 = Solve_RO(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat, P_dag,psi_lb,psi_ub,phi_lb,phi_ub)
        rev_RO_2, price_RO_2 = compute_oof(X_RO_2, A_true, B_true, Obs_Feat, P_dag)
        println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb,",rev_RO = ",round(rev_RO_2,digits=6),",price_RO = ",price_RO_2)
        RST_RO_Two_Side["iter=$(iter)_Rev"] = rev_RO_2
        RST_RO_Two_Side["iter=$(iter)_Price"] = price_RO_2
    end
    return RST_RO_Two_Side
end

In [ ]:
function Run_Two_Side_RO_Cross_validation(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef_all,phi_coef_all)
    RST_RO_Two_Side = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        A_lb = A_true .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_true .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_true .- bd_coef .* abs.(ones(N,N));
        B_ub = B_true .+ bd_coef .* abs.(ones(N,N));

        psi_ub = 0.0
        phi_ub = 0.0
        obj_opt = -100000
        for psi_coef in psi_coef_all
            psi_lb = 0.0 - psi_coef * 10
            for phi_coef in phi_coef_all
                phi_lb = 0.0 - phi_coef * 10

                obj_RO_2,X_RO_2,time_RO_2 = Solve_RO(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat, P_dag,psi_lb,psi_ub,phi_lb,phi_ub)
                println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb,",obj_RO_2 = ",round(obj_RO_2,digits=6))
                if obj_RO_2 > obj_opt
                    obj_opt = obj_RO_2

                    rev_RO_2, price_RO_2 = compute_oof(X_RO_2, A_true, B_true, Obs_Feat, P_dag)
                    println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb,",rev_RO = ",round(rev_RO_2,digits=6),",price_RO = ",price_RO_2)
                    RST_RO_Two_Side["iter=$(iter)_Rev"] = rev_RO_2
                    RST_RO_Two_Side["iter=$(iter)_Price"] = price_RO_2

                end
            end
        end

    end
    return RST_RO_Two_Side
end

## offdiag is zero

In [ ]:
Random.seed!(2)
# 设置参数
N = 3   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "zero"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
iter = 1
Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
A_true = Input_Data["iter=$(iter)_A_true"]
B_true = Input_Data["iter=$(iter)_B_true"]
P_dag = Input_Data["iter=$(iter)_P_dag"]

obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)

In [ ]:
obj_ETO,X_ETO,time_ETO = Solve_ETO_COPT_Version(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
# bd_coef = 0.000001
bd_coef = 0.0
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
# psi_coef = 0.05
# phi_coef = 0.05
# RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
# Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
# println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))

## offdiag is positive

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "positive"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.0
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
# psi_coef = 0.01
# phi_coef = 0.01
# RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
# Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
# println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))

In [ ]:
# psi_coef_all = [0.001,0.005,0.01, 0.02,0.04,0.06,0.08, 0.1]
# phi_coef_all = [0.001,0.005,0.01, 0.02,0.04,0.06,0.08, 0.1]
# Run_Two_Side_RO_Cross_validation(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef_all,phi_coef_all)

## offdiag is negative

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "negative"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.0
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
# psi_coef = 1
# phi_coef = 1
# RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
# Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
# println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))

## offdiag is mix

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "mix"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.0
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
# psi_coef = 1
# phi_coef = 1
# RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
# Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
# println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))